In [1]:
import numpy as np
import torch

Training data

The training data can be represented using 2 matrices: inputs and targets, each with one row per observation, and one column per variable.


In [2]:
# input (temperature, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                  [91, 88, 64],
                  [87, 134, 58],
                  [102, 43, 37],
                  [69, 96, 70]], dtype='float32') # mentioning a data type this way reduces memory usage and holds only floating values in the search

In [4]:
# targets ( apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

We've separated the input and target variables, because we'll operate on them separately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors as follows:


In [5]:
# Convert arrays to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Linear regression model from scratch

The weights and biases (w11, w12,... w23, b1 & b2) can also be represented as matrices, initialized as random values. The first row of w and the first element of b are used to predict the first target variable i.e. yield of apples, and similarly the second for oranges.

Let us create a random values for weights(matrix with 2 rows, 3 cols) & bias(2 values, as its a vector). We've to test these random values in search of better predictions.

In [6]:
# weights and biases for apples and oranges
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True) # in two columns, two biases for two targets
print(w)
print(b)

tensor([[-0.8682, -0.0134, -0.2440],
        [-1.3539, -1.1554, -0.8207]], requires_grad=True)
tensor([ 0.5381, -1.0920], requires_grad=True)


torch.randn creates a tensor with the given shape, with elements picked randomly from a normal distribution with mean 0 and standard deviation 1.

Our model is simply a function that performs a matrix multiplication of the inputs and the weights w (transposed) and adds the bias b (replicated for each observation). We can define the model as follows:


In [7]:
def model(x):
    return x @ w.t() + b

@ is matrix multiplication in pytorch and .t() method will transpose the matrix applied to

In [8]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ -74.2304, -212.6258],
        [ -95.2631, -278.4934],
        [ -90.9450, -321.3029],
        [ -97.6215, -219.2334],
        [ -77.7343, -262.8761]], grad_fn=<AddBackward0>)

Let's compare the predictions of our model with the actual targets.

In [9]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

You can see that there's a huge difference between the predictions of our model, and the actual values of the target variables. Obviously, this is because we've initialized our model with random weights and biases, and we can't expect it to just work.


Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets, using the following method:

    Calculate the difference between the two matrices (preds and targets).
    Square all elements of the difference matrix to remove negative values.
    Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the mean squared error (MSE)

In [10]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

torch.sum returns the sum of all the elements in a tensor, and the .numel method returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.


In [11]:
# Compute loss
loss = mse(preds, targets)
loss

tensor(78084.8438, grad_fn=<DivBackward0>)

Here’s how we can interpret the result: On average, each element in the prediction differs from the actual target by about 145 (square root of the loss 20834). And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. Also, the result is called the loss, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model.


Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have requires_grad set to True.


In [13]:
# Compute gradient
loss.backward()

The gradients are stored in the .grad property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix, with the same dimensions.


In [18]:
# Gradient for weights
print(w)
w.grad

tensor([[-0.8682, -0.0134, -0.2440],
        [-1.3539, -1.1554, -0.8207]], requires_grad=True)


tensor([[-13696.8086, -14972.6885,  -9226.9912],
        [-29435.0391, -32177.2168, -19800.4043]])

The loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases.

If a gradient element is positive:

    increasing the element's value slightly will increase the loss.
    decreasing the element's value slightly will decrease the loss

postive-gradient

The loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases.

If a gradient element is positive:

    increasing the element's value slightly will increase the loss.
    decreasing the element's value slightly will decrease the loss

If a gradient element is negative:

    increasing the element's value slightly will decrease the loss.
    decreasing the element's value slightly will increase the loss.
The loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the loss function w.r.t. the weights and biases.

If a gradient element is positive:

    increasing the element's value slightly will increase the loss.
    decreasing the element's value slightly will decrease the loss.
    
If a gradient element is negative:

    increasing the element's value slightly will decrease the loss.
    decreasing the element's value slightly will increase the loss.

The increase or decrease in loss by changing a weight element is proportional to the value of the gradient of the loss w.r.t. that element. This forms the basis for the optimization algorithm that we'll use to improve our model.

Before we proceed, we reset the gradients to zero by calling .zero_() method. We need to do this, because PyTorch accumulates, gradients i.e. the next time we call .backward on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results.


In [22]:
w.grad.zero_()
b.grad.zero_()
print(w)
print(b)

tensor([[-0.8682, -0.0134, -0.2440],
        [-1.3539, -1.1554, -0.8207]], requires_grad=True)
tensor([ 0.5381, -1.0920], requires_grad=True)


Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:

    Generate predictions

    Calculate the loss

    Compute gradients w.r.t the weights and biases

    Adjust the weights by subtracting a small quantity proportional to the gradient

    Reset the gradients to zero

Let's implement the above step by step.


In [23]:
# Generate predcitions
preds = model(inputs)
preds

tensor([[ -74.2304, -212.6258],
        [ -95.2631, -278.4934],
        [ -90.9450, -321.3029],
        [ -97.6215, -219.2334],
        [ -77.7343, -262.8761]], grad_fn=<AddBackward0>)

Note that predictions are same as before. Since, we didn't make any changes to our weights and biases, same with the loss and the gradient as unchanged.

In [24]:
# Calculate loss
loss = mse(preds, targets)
loss

tensor(78084.8438, grad_fn=<DivBackward0>)

In [25]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-13696.8086, -14972.6885,  -9226.9912],
        [-29435.0391, -32177.2168, -19800.4043]])
tensor([-163.3589, -350.9063])


Finally, we update the weights and biases using the gradients computed above.

In [26]:
# Adjust the weights and reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

A few things to note above:

    We use torch.no_grad to indicate to PyTorch that we shouldn't track, calculate or modify gradients while updating the weights and biases.

    We multiply the gradients with a really small number (10^-5 in this case), to ensure that we don't modify the weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient. This number is called the learning rate of the algorithm.

    After we have updated the weights, we reset the gradients back to zero, to avoid affecting any future computations.

Let's take a look at the new weights and biases.


In [27]:
print(w)
print(b)

tensor([[-0.7312,  0.1363, -0.1517],
        [-1.0595, -0.8336, -0.6227]], requires_grad=True)
tensor([ 0.5397, -1.0885], requires_grad=True)


In [28]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(52674.6641, grad_fn=<DivBackward0>)

We have already achieved a significant reduction in the loss, simply by adjusting the weights and biases slightly using gradient descent.

Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [29]:
# Training for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

Once again, lets verify that our loss now.

In [30]:
# Compute loss
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(63.8710, grad_fn=<DivBackward0>)

As you can see, the loss is now much lower than what we started out with. Let's look at the model's predictions and compare them with the targets

In [31]:
# Predictions
preds

tensor([[ 58.7600,  73.1387],
        [ 80.3731,  98.7008],
        [120.3015, 132.8850],
        [ 29.5265,  54.1145],
        [ 93.9097, 105.5216]], grad_fn=<AddBackward0>)

In [32]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The prediction are now quite close to the target variables, and we can get even better results by training for a few more epochs. Lets try for 100 more

In [34]:
# For 100 more epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [35]:
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(31.2406, grad_fn=<DivBackward0>)

In [36]:
# Lets do for 1000 epochs now..It will slowly decrease
for i in range(1000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [38]:
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(3.5313, grad_fn=<DivBackward0>)

In [40]:
tuple = (preds, targets)
tuple

(tensor([[ 57.3724,  70.3037],
         [ 81.1872,  99.3396],
         [120.6807, 135.9898],
         [ 21.6383,  37.8947],
         [ 99.9292, 116.0982]], grad_fn=<AddBackward0>),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

In [41]:
# Pushing further
for i in range(10000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [42]:
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(0.5080, grad_fn=<DivBackward0>)

Great, Loss is almost zero. This could also be overfitting. Lets explore with a test data that we generate and see how it performs then.